In [1]:
import argparse
import sys

import torch

from model import load_llama_model

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# torch.cuda.set_device("cuda:1")
model, tokenizer = load_llama_model(model_uri="GreenBitAI/LLaMA-7B-2bit", cache_dir="./cache", groupsize=32, double_groupsize=-1, v1=True, bits=2, half=True, asym=True)
model.eval()
model = model.cuda()

Loading Model ...


/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


Total 2.80 Gib VRAM used.
Converted as Half.


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loaded the model in 3.18 seconds.


In [3]:
prompt = '''The meaning of life is'''
batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
batch = {k: v.to(0) for k, v in batch.items()}

In [4]:
from datasets import load_dataset
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [5]:
generated = model.generate(
            inputs=batch["input_ids"],
            do_sample=False,
            use_cache=True,
            repetition_penalty=1.5,
            max_new_tokens=256,
            temperature=0.9,
            top_p=0.95,
            top_k=20,
            return_dict_in_generate=True,
            # output_attentions=False,
            # output_hidden_states=False,
            # output_scores=False
        )

/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [5]:
input_ids = encodings.input_ids[..., :2048].cuda(
)

In [8]:
generated = model(
            input_ids=input_ids,
            labels=input_ids.clone(),
            # do_sample=False,
            use_cache=True,
            # repetition_penalty=1.5,
            # max_new_tokens=256,
            # temperature=0.9,
            # top_p=0.95,
            # top_k=20,
            return_dict=True,
            # output_attentions=False,
            # output_hidden_states=False,
            # output_scores=False
        )

In [9]:
generated.loss

tensor(1.7294, device='cuda:0', grad_fn=<NllLossBackward0>)

In [23]:
model.vocab_size

32000

In [15]:
batch['input_ids']

tensor([[   0,  450, 6593,  310, 2834,  338]], device='cuda:0')

In [22]:
generated.logits.shape

torch.Size([1, 5, 32000])

In [6]:
generated.past_key_values[0][0].dtype

torch.float16

In [19]:
tokenizer('''The meaning of life is''')

{'input_ids': [0, 450, 6593, 310, 2834, 338], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [30]:
model.seqlen

2048

In [31]:
generated.past_key_values[0][0].dtype

torch.float16

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch
model_id = "huggyllama/llama-7b"
tokenizer2 = AutoTokenizer.from_pretrained(model_id)
model_high = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16,)

/home/v-yuanqwang/miniconda3/envs/llama3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model_high.config.max_sequence_length

2048

In [10]:
model_high.config

LlamaConfig {
  "_name_or_path": "huggyllama/llama-7b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.40.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [26]:
encodings = tokenizer2('''The meaning of life is to get more money''', return_tensors='pt')
input_ids = encodings.input_ids.cuda()
input_ids

tensor([[   1,  450, 6593,  310, 2834,  338,  304,  679,  901, 6909]],
       device='cuda:0')

In [25]:
batch = tokenizer('''The meaning of life is to get more money''', return_tensors="pt")
batch

{'input_ids': tensor([[   0,  450, 6593,  310, 2834,  338,  304,  679,  901, 6909]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
generated = model_high(
            input_ids=input_ids,
            use_cache=True,
            return_dict=True,
            # output_attentions=False,
            # output_hidden_states=False,
            # output_scores=False
        )

In [28]:
type(generated.past_key_values)

tuple